In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
try: 
    from googlesearch import search 
except ImportError:  
    print("No module named 'google' found")
from selenium import webdriver
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')  # Last I checked this was necessary.
CHROMEDRIVER_PATH = "E:/Solve for Tomorrow (6292020)/Selenium/chromedriver.exe"
driver = webdriver.Chrome(CHROMEDRIVER_PATH, options=options)

No module named 'google' found


In [2]:
driver.get("https://www.easemytrip.com/")
driver.maximize_window()

In [19]:
# content = driver.page_source
# soup = BeautifulSoup(content)
# soup.findAll('div', attrs={'class':'middle_sec'})

In [3]:
button = driver.find_element_by_xpath('//*[@id="myTopnav"]/div[1]/ul/li[1]/a')
webdriver.ActionChains(driver).move_to_element(button).perform()
time.sleep(2)
webdriver.ActionChains(driver).click(button).perform()

In [7]:
# driver.find_element_by_xpath('/html/body/div[13]/div/div/ul')
allElements = driver.findElements(By.xpath("/html/body/div[13]/div/div/ul/li")); 

# for (WebElement element: allElements) {
#       System.out.println(element.getText());
# }
allElements

AttributeError: 'WebDriver' object has no attribute 'findElements'

In [23]:
# #clicking view more
# button = driver.find_elements_by_class_name("btn-white-green")[0]
# for i in range(15):
#     webdriver.ActionChains(driver).move_to_element(button).perform()
#     time.sleep(2)
#     webdriver.ActionChains(driver).click(button).perform()

IndexError: list index out of range

In [ ]:
# speakers=[] #List to speaker information
# speaker_image_urls = []
# content = driver.page_source
# soup = BeautifulSoup(content)
# for div in soup.findAll('div', attrs={'class':'speaker'}):
#     speaker = div.find('div', attrs={'class':'speaker__content__inner'})
#     speaker_image = div.find('div', attrs={'class':'speaker__image'})
#     speakers.append(speaker.get_text(strip=True, separator="$").split('$'))
#     speaker_image_urls.append(speaker_image['style'][23:-3])
# print('No. of speakers: ', len(speakers))
# print(speakers[0])
# print(speaker_image_urls[0])

In [ ]:
# names = []
# Designations = []
# companies = []
# for speaker in speakers:
#     names.append(speaker[0])
#     Designations.append(speaker[1])
#     companies.append(speaker[2])

In [ ]:
# linkedin_urls = []
# for speaker in speakers:
#     query = speaker[0] + ' ' + speaker[1] + ' ' + speaker[2] + ' linkedin'
# #     query = 'Brad Smith President Microsoft Corporation linkedin'
#     for j in search(query, tld="co.in", num=10, stop=1, pause=2):
#         linkedin_urls.append(j)

In [ ]:
# driver.execute_script("window.scrollBy(0,0)","")
# time.sleep(2)
# short_biographies = []
# for i in range(len(names)):
#     el = driver.find_elements_by_class_name("speaker")
#     webdriver.ActionChains(driver).move_to_element(el[i]).perform()
#     time.sleep(1)
#     webdriver.ActionChains(driver).click(el[i]).perform()
#     content = driver.page_source
#     soup = BeautifulSoup(content)
#     for obj in soup.findAll('div', attrs={'class':'modal-content'}):
#         short_biographies.append(obj.find_all("p")[0].text)
#     close_button = driver.find_elements_by_class_name("close-btn")
#     print(i,' \n')
#     webdriver.ActionChains(driver).click(close_button[0]).perform()

In [ ]:
# len(short_biographies)

In [ ]:
# df = pd.DataFrame({'Speaker Name':names,
#                    'Display Pic url':speaker_image_urls,
#                    'Designation':Designations,
#                    'Company':companies,
#                    'Short bio':short_biographies,
#                    'Linkedin': linkedin_urls}) 
# df.to_csv('ID01.csv', index=False, encoding='utf-8')